# Mask Detection 

**EXTRACTING FACES**

In [2]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
# resize pixels to the model size
    image = Image.fromarray(pixels)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array

In [3]:
def load_face(dir):
    faces = list()
    # enumerate files
    for filename in os.listdir(dir):
        path = dir + filename
        face = extract_face(path)
        faces.append(face)
    return faces

def load_dataset(dir):
    # list for faces and labels
    X, y = list(), list()
    for subdir in os.listdir(dir):
        path = dir + subdir + '/'
        faces = load_face(path)
        labels = [subdir for i in range(len(faces))]
        # print("loaded %d sample for class: %s" % (len(faces),subdir) ) # print progress
        X.extend(faces)
        y.extend(labels)
    return np.asarray(X), np.asarray(y)

**LOAD AND  TRAIN FOR FACE RECOGNITION**

In [6]:
# load train dataset
trainX, trainy = load_dataset('../input/fyprmfrd/RMFRD/AFDB_face_datset/')
print(trainX.shape, trainy.shape)

In [7]:
# load test dataset
testX, testy = load_dataset('../input/fyprmfrd/RMFRD/AFDB_masked_face_dataset/')
print(testX.shape, testy.shape)

**Compress and Save Train and Test Dataset**
 After face extraction process, train and dataset occupied memory and there was a memory error. The reason of this process is to deal with this issue .

In [8]:
# save and compress the dataset for further use
np.savez_compressed('new_modified_masked_face.npz', trainX, trainy, testX, testy)
print('Done with Compression')
del trainX, trainy, testX, testy

In [9]:
data = np.load('/kaggle/working/new_modified_masked_face.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
del data

In [10]:
trainx, valid = train_test_split(trainX, test_size=0.4, random_state=42, shuffle=True)
del trainX
print('Split Training Data')

In [11]:
print("number of image in train dataset : %s" %(len(trainx)))

print("number of image in train dataset : %s" %(len(valid)))

In [12]:
y_train, y_valid = train_test_split(trainy, test_size=0.4, random_state=42, shuffle=True)
del trainy
print('Split Training Data')

In [13]:
print("number of image in train dataset : %s" %(len(y_train)))

print("number of image in train dataset : %s" %(len(y_valid)))

In [14]:
# save and compress the dataset for further use
np.savez_compressed('modified_extracted_masked_unmasked.npz', trainx, y_train, valid, y_valid,testX, testy)
del trainx, y_train, valid, y_valid, testX, testy
print('Done Compressing')

In [15]:
data = np.load('/kaggle/working/modified_extracted_masked_unmasked.npz')
print(data.files)
trainx, y_train, valid, y_valid,testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3'], data['arr_4'], data['arr_5']
print('Loaded: ', trainx.shape, y_train.shape, valid.shape, y_valid.shape,testX.shape, testy.shape)
del data

**FACE EMBEDDINGS WITH FACENET**


In [16]:
facenet_model = load_model('/kaggle/input/facenet/keras-facenet/model/facenet_keras.h5')
print('Loaded Model')

In [17]:
print('Loaded: ', trainx.shape, y_train.shape, valid.shape, y_valid.shape,testX.shape, testy.shape)

In [18]:
def get_embedding(model, face):
    # scale pixel values
    face = face.astype('float32')
    # standardization
    mean, std = face.mean(), face.std()
    face = (face-mean)/std
    # transfer face into one sample (3 dimension to 4 dimension)
    sample = np.expand_dims(face, axis=0)
    # make prediction to get embedding
    yhat = model.predict(sample)
    return yhat[0]

**CONVERT EACH FACE IN THE TRAINSET INTO EMBEDDING**


In [19]:
emdTrainX = list()
for face in trainx:
    emd = get_embedding(facenet_model, face)
    emdTrainX.append(emd)
emdTrainX = np.asarray(emdTrainX)
print(emdTrainX.shape)
embValid = list()
for face in valid:
    emd = get_embedding(facenet_model,face)
    embValid.append(emd)
embValid = np.asarray(embValid)
print(embValid.shape)

In [23]:
emdTestX = list()
for face in testX:
    emd = get_embedding(facenet_model, face)
    emdTestX.append(emd)
emdTestX = np.asarray(emdTestX)
print(emdTestX.shape)

**COMPRESSS AND SAVE TRAIN AND TEST EMBEDDINGS**


In [24]:
# save arrays to one file in compressed format
np.savez_compressed('embeddings_masked.npz', emdTrainX, y_train, embValid, y_valid, emdTestX, testy)
del emdTrainX, y_train, embValid, y_valid, emdTestX, testy
print('Here')

In [25]:
data = np.load('/kaggle/working/embeddings_masked.npz')
print(data.files)
emdTrainX, y_train, embValid, y_valid, emdTestX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3'], data['arr_4'], data['arr_5']

In [26]:
print('Loaded: ', emdTrainX.shape, y_train.shape, embValid.shape, y_valid.shape, emdTestX.shape, testy.shape)

**LABELLED ENCODING**


In [27]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import pickle
print("Dataset: train=%d,validation = %d, test=%d" % (emdTrainX.shape[0],embValid.shape[0] ,emdTestX.shape[0]))
# normalize input vectors
in_encoder = Normalizer(norm='l2')
emdTrainX_norm = in_encoder.transform(emdTrainX)
embValid_norm = in_encoder.transform(embValid)
emdTestX_norm = in_encoder.transform(emdTestX)
# label encode targets
out_encoder = LabelEncoder()
encoder_arr = np.append (y_train, 'wangnan')
out_encoder.fit(encoder_arr)

**ENCODING TRAINY AND TESTY WITH FITTED ENCODER**



In [28]:
trainy_enc = out_encoder.transform(y_train)
y_valid_enc = out_encoder.transform(y_valid)
testy_enc = out_encoder.transform(testy)

**Face Classification with SVC**

In [29]:
model = SVC(kernel='linear', probability=True)
model.fit(emdTrainX_norm, trainy_enc)

In [30]:
# predict
yhat_valid = model.predict(embValid_norm)
yhat_test = model.predict(emdTestX_norm)
# score
score_valid = accuracy_score(y_valid_enc, yhat_valid)
score_test = accuracy_score(testy_enc, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_valid*100, score_test*100))

In [31]:
#Save the model
filename = 'linear.sav'
pickle.dump(model, open(filename, 'wb'))

In [32]:
loaded_model = pickle.load(open('linear.sav', 'rb'))

**TRAIL**

In [33]:
import numpy as np
import pandas as pd
import os
from sklearn.utils import shuffle
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
from keras.models import load_model
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras;    from tensorflow.keras import layers;
from tensorflow.keras import metrics

In [34]:
no_mask_path = '../input/fyprmfrd/RMFRD/AFDB_face_datset/'

#creating dataframe with given data

image_no_mask = []
target_no_mask = []

for i in os.listdir(no_mask_path):
    
    for j in os.listdir(no_mask_path + "/" + i):
        pic = os.path.join(no_mask_path + "/" + i, j)
    
        image_no_mask.append(pic)
        target_no_mask.append("without_mask")
        
        #print(pic)
    

mask_path = '../input/fyprmfrd/RMFRD/AFDB_masked_face_dataset'

image_mask = []
target_mask = []

for i in os.listdir(mask_path):
    
    for j in os.listdir(mask_path + "/" + i):
    
        pic = os.path.join(mask_path + "/" + i, j)
    
        image_mask.append(pic)
        target_mask.append("mask")
    

mask = pd.DataFrame()
mask['image'] = image_mask
mask['target'] = target_mask

no_mask = pd.DataFrame()
no_mask['image'] = image_no_mask
no_mask['target'] = target_no_mask

data = pd.concat([mask, no_mask], axis=0, ignore_index=True)
data = shuffle(data)

data

In [35]:
sns.countplot(data['target'])

In [36]:
train_data = data.iloc[0:3000, ]
val_data = data.iloc[3000:4000, ]
test_data = data.iloc[4000: , ]

In [37]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode = "nearest")

train_generator = train_datagen.flow_from_dataframe(dataframe = train_data,
                                                    x_col = "image",
                                                    y_col = "target",
                                                    target_size = (150, 150),
                                                    batch_size = 32,
                                                    class_mode = "binary",
                                                    validate_filenames=False)

val_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = val_datagen.flow_from_dataframe(dataframe = val_data,
                                                x_col = "image",
                                                y_col = "target",
                                                target_size = (150, 150),
                                                batch_size = 32,
                                                class_mode = "binary",
                                                validate_filenames=False)


test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(dataframe = test_data,
                                                  x_col = "image",
                                                  y_col = "target",
                                                  target_size = (150, 150),
                                                  class_mode = "binary",
                                                  batch_size = 32,
                                                  validate_filenames=False)

In [38]:
from keras import layers
from keras import models
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input

vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

for layer in vgg19.layers:
    layer.trainable = False
    
model = models.Sequential()
model.add(vgg19)
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [39]:
from tensorflow.keras import optimizers
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(learning_rate=0.0001), 
              metrics=['acc', keras.metrics.Precision(), keras.metrics.Recall()])

In [40]:
model.fit(train_generator, epochs = 1, validation_data = val_generator)

In [41]:
test_loss, test_acc, test_precision, test_recall = model.evaluate(test_generator, steps=10)

In [42]:
from random import choice

for i in range(20):
    
    
    # select a random face from test set
    selection = choice([i for i in range(trainx.shape[0])])
    random_face = trainx[selection]
    random_face_emd = emdTrainX_norm[selection]
    random_face_class = trainy_enc[selection]
    random_face_name = out_encoder.inverse_transform([random_face_class])
    
    print(random_face.shape)
    
    # prediction for wearing mask or not
    img = np.resize(random_face, (128, 128, 3))
    img = np.array(img)
    img = img / 255.0
    img = img.reshape(-1, 128, 128, 3)
    
    pred_prob = model.predict(img)[0]
    pred_class = list(pred_prob).index(max(pred_prob))
    
    if pred_class == 0:
        mask_class = "Wearing Mask"
    
    else:
        mask_class = "Not Wearing Mask"
    
    
    # prediction for the face
    samples = np.expand_dims(random_face_emd, axis=0)
    yhat_class = loaded_model.predict(samples)
    yhat_prob = loaded_model.predict_proba(samples)
    class_index = yhat_class[0]
    
    if class_index <= 460:
        # get name
        class_probability = yhat_prob[0,class_index] * 100
        predict_names = out_encoder.inverse_transform(yhat_class)
        
        print('Mask Detection: %s (Class = %d)' % (mask_class, pred_class))
        print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
        print('Expected: %s' % random_face_name[0])
        
        # plot face
        plt.imshow(random_face)
        title = '%s (%.3f)' % (predict_names[0], class_probability)
        plt.title(title)
        plt.show()